In [5]:
import xml.etree.ElementTree as ET
from pprint import pprint
import re
from collections import defaultdict
import csv
import codecs
# import cerberus

filename = 'san-francisco_california.osm'

OSM_PATH = 'san-francisco_california.osm'

osm_file = 'san-francisco_california.osm'

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"


In [6]:
sample_file = "sf_sample.osm"

k = 5

def get_element(osm_file):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            yield elem
            root.clear()


with open(sample_file, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')

    for i, element in enumerate(get_element(osm_file)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>')

In [28]:
def shape_element(filename):
    counter = 0 
    ways = {
        'way': {}
    }
    for elem in get_element(filename):
        if counter < 100:
            if elem.tag == 'way':
                way_attributes = {
                    'id': elem.attrib['id'],
                    'version': elem.attrib['version'],
                    'timestamp': elem.attrib['timestamp'],
                    'changeset': elem.attrib['changeset'],
                    'uid': elem.attrib['uid'],
                    'user': elem.attrib['user']
                         }
                counter += 1
                ways['way'].update(way_attributes)
                
#                 way_nodes = {}
#                 node_counter = 0
#                 for child in elem.iter('nd'):
#                     way_node_attributes = {
#                         'way_nodes': [{
#                         'id': elem.attrib['id'],
#                         'node_id': child.attrib['ref'],
#                         'position': node_counter
#                           }]
#                     }
#                     way_nodes.update(way_node_attributes)
#                     node_counter += 1
#                 counter += 1
#                 ways.add(way_nodes)
                
#                 way_tags = []
#                 for child in elem.iter('tag'):
#                     tag_attributes = {
#                         'id': elem.attrib['id'],
#                         "key": child.attrib['k'],
#                         'value': child.attrib['v']
#                             }
#                     way_tags.append(tag_attributes)
#                 ways.append('way_tags:')
#                 ways.append(way_tags)
   
    return ways




shaped_element = shape_element(sample_file)

pprint(shaped_element)

    

{'way': {'changeset': '4244079',
         'id': '6326935',
         'timestamp': '2010-03-27T09:35:42Z',
         'uid': '20587',
         'user': 'balrog-kun',
         'version': '2'}}


In [15]:
# lower = re.compile(r'^([a-z]|_)*$')
# lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
# problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')



expected = ['Street', 'Avenue', 'Drive', 
            'Road', 'Boulevard', 'Court',
            'Place', 'Highway', 'Way', 
            'Alley', 'Plaza', 'Circle',
            'Park', 'Lane', 'Parkway', 
            'Path', 'Road', 'Terrace',
            'Center', 'Square', "North", 
            "East", "West", "South"]

def audit_street_names(filename):
    unexpected_street_names = defaultdict(set)
    for elem in get_element(filename):
        for tag in elem.iter('tag'):
            if tag.attrib['k'] == 'addr:street':
                street_name = tag.attrib['v'].split()
                street_name_suffix = street_name[-1]
                for word in street_name:
                     if street_name_suffix not in expected:
                        unexpected_street_names[tag.attrib['k']].add(tag.attrib['v'])
                        
                        
    return unexpected_street_names

audited_street_names = audit_street_names(sample_file)

pprint(audited_street_names)

In [32]:
mapping = { 
            "St": "Street",
            "St.": "Street",
            "STREET": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr.": "Drive",
            "Dr": "Drive",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Ehs": "EHS",
            "Trl": "Trail",
            "Cir": "Circle",
            "Cir.": "Circle",
            "Ct": "Court",
            "Ct.": "Court",
            "Crt": "Court",
            "Crt.": "Court",
            "By-pass": "Bypass",
            "N.": "North",
            "N": "North",
            "E.": "East",
            "E": "East",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West"
          }


expected = ['Street', 'Avenue', 'Drive', 
            'Road', 'Boulevard', 'Court',
            'Place', 'Highway', 'Way', 
            'Alley', 'Plaza', 'Circle',
            'Park', 'Lane', 'Parkway', 
            'Path', 'Road', 'Terrace',
            'Center', 'Square', "North", 
            "East", "West", "South"]



def fix_street_names(filename):
    counter = 0
    fixed_street_names = []
    for elem in get_element(filename):
        for tag in elem.iter('tag'):
            if tag.attrib['k'] == 'addr:street':
                street_name = tag.attrib['v'].split()
                street_name_suffix = street_name[-1]
                for word in street_name:
                     if street_name_suffix not in expected and counter < 100:
                        for abbreviation, correct_spelling in mapping.items():
                            counter += 1
                            if street_name_suffix == abbreviation:
                                street_name_suffix_fixed = correct_spelling
                                fixed_street_names.append(street_name)              
                        

    return fixed_street_names  

fixed_street_names = fix_street_names(sample_file)

pprint(fixed_street_names)
    

[['Park', 'St'], ['Park', 'St']]


In [57]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

street_map = { 
            "St": "Street",
            "St.": "Street",
            "STREET": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr.": "Drive",
            "Dr": "Drive",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Trl": "Trail",
            "Cir": "Circle",
            "Cir.": "Circle",
            "Ct": "Court",
            "Ct.": "Court",
            "Crt": "Court",
            "Crt.": "Court",
            "N.": "North",
            "N": "North",
            "E.": "East",
            "E": "East",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West"
          }

expected = ['Street', 'Avenue', 'Drive', 
            'Road', 'Boulevard', 'Court',
            'Place', 'Highway', 'Way', 
            'Alley', 'Plaza', 'Circle',
            'Park', 'Lane', 'Parkway', 
            'Path', 'Road', 'Terrace',
            'Center', 'Square']


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    street_types = defaultdict(set)
    for elem in get_element(filename):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

audited = audit(sample_file)

pprint(audited)



defaultdict(<class 'set'>,
            {'1': {'W Of Us 101 @ Jct Sr 1'},
             '100': {'Woodside Road, Suite 100'},
             '105': {'N California Blvd #105', 'Grand Avenue #105'},
             '10675': {'10675'},
             '12180142': {'12180142'},
             '122°29\'07.1"W': {'37°42\'46.0"N 122°29\'07.1"W'},
             '15': {'Doolittle Dr, Suite 15'},
             '151': {'Brannan St #151'},
             '155': {'Woodside Road, Suite 155', 'University Ave #155'},
             '15th': {'15th'},
             '2': {'San Francisco Bicycle Route 2'},
             '203': {'Bartlett Street #203'},
             '3': {'University Dr. Suite 3'},
             '3.2': {'ALA 84 PM 3.2'},
             '302': {'Mission Blvd #302'},
             '3500': {'Sansome St #3500'},
             '3658': {'Market Street Suite 3658'},
             '39': {'Pier 39'},
             '4': {'Menlo Ave  # 4'},
             '4.5': {'SF 80 PM 4.5'},
             '404': {'16th St #404'},
            

In [12]:
#Helper Functions

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


NameError: name 'SCHEMA' is not defined

In [13]:
#Main Function

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        
        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


SQL Schema
https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f

Problem Encountered: Dead URLs


In [23]:
def audit_zipcode(filename):
    ways = []
    for elem in get_element(filename):
        if elem.tag == 'way':
            for child in elem.iter('tag'):
                if child.attrib['k'] == "addr:postcode":
                    if len(child.attrib['v']) != 5:
                        ways.append(child.attrib['v'])

    return ways
audited_zipcode = audit_zipcode(sample_file)

pprint(audited_zipcode)

['94002-3585', 'CA 94544', '94301-2019', '94117-9991', '94118-1316']


In [31]:
def fix_zip_codes(filename):
    zip_codes = []
    for elem in get_element(filename):
        if elem.tag == 'way':
            for child in elem.iter('tag'):
                if child.attrib['k'] == "addr:postcode":
                    zip_code = child.attrib['v']
                    if len(zip_code) == 10:
                        zip_code = zip_code[:-5]
                        zip_codes.append(zip_code)
                    elif len(zip_code) != 5:
                        zip_codes.append(zip_code)
    return zip_codes
                        
fixed_zip_code = fix_zip_codes(sample_file)

pprint(fixed_zip_code)

['94002', 'CA 94544', '94301', '94117', '94118']


In [6]:
#We are now going a little bit deeper in our code and printing the attributes of each of the elements


def audit_tags(filename):
    attributes = defaultdict(set)
    for elem in get_element(filename):
        for tag in elem.iter('tag'):
            attributes[tag.attrib['k']].add(tag.attrib['v'])
            
    return attributes

sf_attributes = audit_tags(sample_file)
print(len(sf_attributes))

pprint(sf_attributes)


In [4]:
def audit_ways(filename):
    
    ways = []
    for elem in get_element(filename):
        for tag in elem.iter('way'):
                attributes = {
                    "id": tag.attrib['id'],
                    "version": tag.attrib['version'],
                    'timestamp': tag.attrib['timestamp'],
                    'changeset': tag.attrib['changeset'],
                    'uid': tag.attrib['uid'],
                    'user': tag.attrib['user']
                }
                ways.append(tag)
                ways.append(attributes)
    return ways

sf_attributes = audit_ways(sample_file)

pprint(sf_attributes)

In [3]:
#This code gives us a high-level overview of our file and counts the parent and child elements

def count_tags(filename):
    tags_dict = {}
    for elem in get_element(filename):
        if elem.tag in tags_dict:
            tags_dict[elem.tag] += 1
        elif elem.tag not in tags_dict:
            tags_dict[elem.tag] = 1
             
    return tags_dict

counted_tags = count_tags(sample_file)

pprint(counted_tags)
    

{'bounds': 1,
 'member': 21131,
 'nd': 3110937,
 'node': 1311766,
 'osm': 1,
 'relation': 1298,
 'tag': 848657,
 'way': 162002}


In [16]:
def count_users(filename):
    unique_users = {}
    for elem in get_element(filename):
        for tag in elem.iter('way'):
            if tag.attrib['user'] not in unique_users:
                unique_users[tag.attrib['user']] = 1
            else:
                unique_users[tag.attrib['user']] += 1
    
    num_unique_users = len(unique_users)
    print (num_unique_users)
    
    return unique_users

count_users(filename);

1764


Resources:

https://discussions.udacity.com/t/p3-openstreetmap-overview/172045/2<br>
https://discussions.udacity.com/t/final-project-need-help-getting-started/181525/60<br>
https://discussions.udacity.com/t/count-tags-function-implementation/264133/2<br>
https://discussions.udacity.com/t/system-ram-goes-to-99-while-wrangling/182980/7<br>
https://wiki.openstreetmap.org/wiki/Elements<br>
http://effbot.org/zone/element.htm<br>
https://stackoverflow.com/questions/15881497/retrieving-xml-attribute-values-using-python-iterparse<br>
https://discussions.udacity.com/t/elements-and-attributes/267615<br>
https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md<br>
http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python<br>
https://stackoverflow.com/questions/15181867/understanding-the-set-function<br>
https://discussions.udacity.com/t/printing-tags-for-keys-and-values/273966/3<br>
https://stackoverflow.com/questions/5518435/python-fastest-way-to-create-a-list-of-n-lists<br>
https://discussions.udacity.com/t/creating-a-list-of-lists-of-ways-tags/274031<br>
https://discussions.udacity.com/t/help-with-count-users-function/274167<br>
https://discussions.udacity.com/t/help-with-audit-street-names-function/274268<br>
https://discussions.udacity.com/t/feedback-on-audit-street-names-method/275561<br>
https://discussions.udacity.com/t/action-plan-for-p3/231324<br>

